In [ ]:
import bz2
import numpy as np

# Ubicación del Word-Embedding en español utilizado.
file_path ='./SBW-vectors-300-min5.txt.bz2'

# Inicialización del diccionario para el embedding index.
embedding_index = {}

with bz2.open(file_path, 'rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]  # la palabra es el primer valor en cada línea
        vector = np.asarray(values[1:], dtype='float32')  # el resto de la línea es el vector
        embedding_index[word] = vector


In [ ]:
import pandas as pd

# Ubicación del dataset utilizado.
data_path = '../Datos/TRAIN.csv'
df=pd.read_csv(data_path)

In [ ]:
df

,Frases,Tipo texto
0,"En el tribunal, el juez dictó una sentencia en...",IA
1,"El médico, en su consultorio, realizó un diagn...",IA
2,La densidad de la mama es la cantidad relativa...,Humano
3,"El bosque, un tapiz de árboles y hojas, era un...",IA
4,La ética de la virtud se centra en el desarrol...,IA
...,...,...
2129,De 1990 a 2008-2012 Alemania ya contribuyó con...,Humano
2130,"La intimidad de todas las caras, el sonido del...",Humano
2131,Conozco a estas dos personas desde hace un tie...,Humano
2132,"WASHINGTON, DC – En menos de 80 días, los líde...",Humano


In [ ]:
MAX_TEXT_LENGTH = 40 # Constante fijada de número de palabras máximo por texto.
EMBEDDING_DIM=300 # La dimensión de los vectores del Embedding utilizado es 300.

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Tokenización del texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Frases)
vocab_size = len(tokenizer.word_index)

sequences = tokenizer.texts_to_sequences(df.Frases)
data = pad_sequences(sequences, maxlen=MAX_TEXT_LENGTH)

# Inicialización de la matriz del embedding
embedding_matrix = np.zeros((vocab_size + 1, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if i < vocab_size:
        if embedding_vector is not None:
            # Palabras no encontradas en el embedding index serán todo ceros
            embedding_matrix[i] = embedding_vector


In [ ]:
# Exportamos el Tokenizer para utilizarlo en otros ficheros.
import io
from json import dumps
tokenizer_json = tokenizer.to_json()
with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(dumps(tokenizer_json, ensure_ascii=False))

In [ ]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.029648,  0.011336,  0.019949, ..., -0.128057, -0.004917,
         0.062628],
       [-0.022313,  0.022251,  0.036704, ..., -0.087151, -0.020682,
         0.033452],
       ...,
       [ 0.057928, -0.02633 , -0.025191, ...,  0.04308 , -0.0568  ,
         0.01227 ],
       [ 0.029573, -0.071883, -0.053401, ..., -0.090921, -0.049074,
        -0.039884],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]])

In [ ]:
# Para resolver este problema de clasificación binaria mediante redes neuronales codificamos el target como 1 o 0.
df['Tipo texto'] = df['Tipo texto'].map({'Humano': 1, 'IA': 0})

In [ ]:
df

,Frases,Tipo texto
0,"En el tribunal, el juez dictó una sentencia en...",0
1,"El médico, en su consultorio, realizó un diagn...",0
2,La densidad de la mama es la cantidad relativa...,1
3,"El bosque, un tapiz de árboles y hojas, era un...",0
4,La ética de la virtud se centra en el desarrol...,0
...,...,...
2129,De 1990 a 2008-2012 Alemania ya contribuyó con...,1
2130,"La intimidad de todas las caras, el sonido del...",1
2131,Conozco a estas dos personas desde hace un tie...,1
2132,"WASHINGTON, DC – En menos de 80 días, los líde...",1


In [ ]:
# Vamos a dividir en un 80 - 20 para entrenamiento - test.
RANDOM_STATE = 1234 # Semilla para que los resultados sean reproducibles.

from sklearn.model_selection import train_test_split

frase_train, frase_test, tipo_train, tipo_test = train_test_split(data, df['Tipo texto'], test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.optimizers import Adam

# Creación del modelo de red neuronal
model = Sequential([
    Embedding(input_dim=vocab_size + 1,
              output_dim=EMBEDDING_DIM,
              weights=[embedding_matrix],
              input_length=MAX_TEXT_LENGTH,
              trainable=False),
    LSTM(EMBEDDING_DIM, batch_input_shape=(None, MAX_TEXT_LENGTH, EMBEDDING_DIM)),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compilación del modelo
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])


# Resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 300)           3031500   
                                                                 
 lstm (LSTM)                 (None, 300)               721200    
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense (Dense)               (None, 50)                15050     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3,767,801
Trainable params: 736,301
Non-tr

In [ ]:
# Entrenamiento del modelo.
history = model.fit(frase_train, tipo_train, epochs=20, batch_size=64, validation_data=(frase_test, tipo_test))

# Guardado del modelo.
model.save('modelo_red_neuronal.keras')

Epoch 1/20


27/27 [==============================] - 6s 174ms/step - loss: 0.7847 - accuracy: 0.5876 - val_loss: 0.6521 - val_accuracy: 0.5293
Epoch 2/20
27/27 [==============================] - 4s 154ms/step - loss: 0.6536 - accuracy: 0.6139 - val_loss: 0.7307 - val_accuracy: 0.5550
Epoch 3/20
27/27 [==============================] - 4s 155ms/step - loss: 0.6618 - accuracy: 0.6520 - val_loss: 0.5880 - val_accuracy: 0.8056
Epoch 4/20
27/27 [==============================] - 4s 160ms/step - loss: 0.3794 - accuracy: 0.8776 - val_loss: 0.4901 - val_accuracy: 0.7822
Epoch 5/20
27/27 [==============================] - 4s 159ms/step - loss: 0.2757 - accuracy: 0.8922 - val_loss: 0.1454 - val_accuracy: 0.9602
Epoch 6/20
27/27 [==============================] - 4s 155ms/step - loss: 0.1653 - accuracy: 0.9455 - val_loss: 0.0908 - val_accuracy: 0.9789
Epoch 7/20
27/27 [==============================] - 4s 158ms/step - loss: 0.1038 - accuracy: 0.9649 - val_loss: 0.0846 - val_accuracy: 0.9696
Epoch 8/20
27/27 

In [ ]:
# Evaluamos el modelo con el dataset proporcionado por Accenture.

data_path = '../Datos/frases.csv'
df_accenture = pd.read_csv(data_path)
sequences = tokenizer.texts_to_sequences(df_accenture.Frases)
data_accenture = pad_sequences(sequences, maxlen=MAX_TEXT_LENGTH)
y_pred = model.predict(data_accenture)

In [ ]:
# La red proporciona valores en coma flotante entre 0 y 1. Consideramos los valores cercanos a 1 como "clasificado Humano"
# y los valores cercanos a 0 como "clasificado IA".
y_pred = ["Humano" if y_pred[i] > 0.5 else "IA" for i in range(len(y_pred))]

In [ ]:
y_pred

[1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(df_accenture["Tipo texto"], y_pred)
matrix

array([[52,  5],
       [ 6, 56]], dtype=int64)

In [ ]:
accuracy = (matrix[0][0] + matrix[1][1]) / matrix.sum()
accuracy

0.907563025210084